In [1]:
import sys
sys.path.append('../')

In [2]:
import numpy as np, pandas as pd
from glob import glob
import shutil, os
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import IncrementalPCA
from tqdm.notebook import tqdm
import seaborn as sns
import PIL.Image as Image
import torch
from torch.utils.data import Dataset, DataLoader
import cv2
import time
import pandas_profiling as pdp
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from utils.util import *
from utils.losses import *
import torch.nn as nn
import transformers as T
import albumentations
import pandas as pd
import cv2
import numpy as np
import timm
import torch.nn as nn
from sklearn import metrics
import torch
from tez.callbacks import EarlyStopping
from tqdm import tqdm
import math
import tez
import torch.optim as optim
import warnings
warnings.simplefilter('ignore')

In [17]:
class CONFIG:
    DATA_PATH = Path('../input/petfinder-pawpularity-score')
    OUTPUT_DIR = Path('../output/2')
    MODEL_NAME = 'efficientnet_b7'
    batch_size = 8
    image_size = 512
    fold = 5
    epoch = 20
    lr = 2e-4
    device='cuda'
    training_step=True
    pretrained=True
    SEED=42
if not os.path.isdir(CONFIG.OUTPUT_DIR):
    os.makedirs(CONFIG.OUTPUT_DIR)
LOGGER = init_logger(OUTPUT_DIR=CONFIG.OUTPUT_DIR)
fix_seed(CONFIG.SEED)

In [18]:
train_df = pd.read_csv(CONFIG.DATA_PATH / 'train.csv')
train_df['path'] = train_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'train'/x)+'.jpg')
train_df['image_size'] = train_df['path'].apply(lambda image_id : Image.open(image_id).size)
train_df['width'] = train_df['image_size'].apply(lambda x: x[0])
train_df['height'] = train_df['image_size'].apply(lambda x: x[1])

test_df = pd.read_csv(CONFIG.DATA_PATH / 'test.csv')
test_df['path'] = test_df['Id'].map(lambda x:str(CONFIG.DATA_PATH/'test'/x)+'.jpg')
test_df['image_size'] = test_df['path'].apply(lambda image_id : Image.open(image_id).size)
test_df['width'] = test_df['image_size'].apply(lambda x: x[0])
test_df['height'] = test_df['image_size'].apply(lambda x: x[1])

train_df.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,path,image_size,width,height
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63,../input/petfinder-pawpularity-score/train/000...,"(405, 720)",405,720
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42,../input/petfinder-pawpularity-score/train/000...,"(1032, 774)",1032,774
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28,../input/petfinder-pawpularity-score/train/001...,"(720, 960)",720,960
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15,../input/petfinder-pawpularity-score/train/001...,"(405, 720)",405,720
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72,../input/petfinder-pawpularity-score/train/001...,"(540, 960)",540,960


In [19]:
train_df = get_train_data(train_df, train_df['Pawpularity'], n_splits = CONFIG.fold, regression=True)

In [6]:
train_aug = albumentations.Compose(
    [
        albumentations.Resize(CONFIG.image_size, CONFIG.image_size, p=1),
        albumentations.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0,
            p=1.0,
        ),
    ],
    p=1.0,
)

In [7]:
class PawpularDataset:
    def __init__(self, df, dense_features, targets, augmentations):
        self.image_paths = df['path'].tolist()
        self.dense_features = dense_features
        self.targets = targets
        if self.targets is None:
            self.targets = torch.ones(len(self.image_paths))
        self.augmentations = augmentations
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, item):
        image = cv2.imread(self.image_paths[item])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        targets = self.targets[item]
        image = np.transpose(image, (2, 0, 1)).astype(np.float32)
        
        features = self.dense_features[item, :]
        
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "features": torch.tensor(features, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float)
        }

In [8]:
class PawpularModel(nn.Module):
    def __init__(self, model_name):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=CONFIG.pretrained, in_chans=3)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(140, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features):
        x = self.model(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        x = self.dense2(x)
        return torch.sigmoid(x.squeeze(1))


In [9]:
def train_fn(train_loader, model, criterion, optimizer, batch_size, epoch, device):
    start = end = time.time()
    losses = AverageMeter()
    model.train()
    for iter, data in tqdm(enumerate(train_loader), total=len(train_loader)):
        optimizer.zero_grad()
        img, feature ,target = data['image'], data['features'], data['targets']
        img = img.to(device)
        feature = feature.to(device)
        target = target.to(device)
        y_preds = model(img, feature)
        loss = criterion(y_preds, target)
        # record loss
        losses.update(loss.item(), batch_size)
        loss.backward()
        optimizer.step()

    return losses.avg

def valid_fn(valid_loader, model, criterion, device):
    start = end = time.time()
    losses = AverageMeter()

    # switch to evaluation mode
    model.eval()
    preds = []

    for iter, data in enumerate(valid_loader):
        img, feature ,target = data['image'], data['features'], data['targets']
        img = img.to(device)
        feature = feature.to(device)
        target = target.to(device)
        batch_size = target.size(0)

        # compute loss
        with torch.no_grad():
            y_preds = model(img, feature)

        loss = criterion(y_preds, target)
        losses.update(loss.item(), batch_size)

        # record score
        preds.append(y_preds.to("cpu").numpy())

    predictions = np.concatenate(preds)
    return losses.avg, predictions

In [10]:
def train_loop(train, fold_):
    LOGGER.info(f"========== fold: {fold_} training ==========")

    # ====================================================
    # Data Loader
    # ====================================================
    model = PawpularModel(model_name=CONFIG.MODEL_NAME)
    model.to(CONFIG.device)
    if torch.cuda.device_count()>1:
        model=nn.DataParallel(model)
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    train_idx = train[train.fold!=fold_].index
    val_idx = train[train.fold ==fold_].index
    train_folds = train.loc[train_idx].reset_index(drop=True)
    valid_folds = train.loc[val_idx].reset_index(drop=True)
    train_dataset = PawpularDataset(
        train_folds, 
        dense_features=train_folds[dense_features].values, targets=train_folds['Pawpularity']/100,
        augmentations=train_aug,
    )
    
    val_dataset = PawpularDataset(
        valid_folds, 
        dense_features=valid_folds[dense_features].values, targets=valid_folds['Pawpularity']/100,
        augmentations=train_aug,
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=True,
        pin_memory=True,
        drop_last=True,
    )
    valid_loader = DataLoader(
        val_dataset,
        batch_size=CONFIG.batch_size,
        shuffle=False,
        pin_memory=True,
        drop_last=False,
    )

    optimizer = optim.Adam(model.parameters(), lr=CONFIG.lr)
    
    criterion = nn.MSELoss()
    metric = RMSE()
    best_score = np.inf
    best_loss = np.inf
    
    for epoch in range(CONFIG.epoch):
        start_time = time.time()
        # train
        avg_loss = train_fn(train_loader, model, criterion, optimizer, CONFIG.batch_size, epoch, CONFIG.device)
#        # eval
        avg_val_loss, preds = valid_fn(valid_loader, model, criterion, CONFIG.device)
        valid_labels = torch.tensor(valid_folds["Pawpularity"].values).float()
        score = metric(preds * 100, valid_labels)
        elapsed = time.time() - start_time
        
        LOGGER.info(
            f"Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s"
        )
        LOGGER.info(f"Epoch {epoch+1} - Score: {score}")
        if score < best_score:
            best_score = score
            LOGGER.info(f"Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model")
            torch.save(
                {"model": model.state_dict(), "preds": preds * 100}, CONFIG.OUTPUT_DIR / f"{CONFIG.MODEL_NAME}_{fold_}_best.pth")
    check_point = torch.load(CONFIG.OUTPUT_DIR / f"{CONFIG.MODEL_NAME}_{fold_}_best.pth")

    valid_folds["preds"] = check_point["preds"]

    return valid_folds

In [11]:
def get_result(result_df):
    metric = RMSE()
    preds = result_df["Pawpularity"].values
    labels = result_df["preds"].values
    score = metric(preds, labels)
    LOGGER.info(f"Score: {score:<.5f}")

In [12]:
def inference():
    predictions = []
    dense_features = [
        'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur'
    ]
    test_dataset = PawpularDataset(
        test_df, 
        dense_features=test_df[dense_features].values, targets=None,
        augmentations=train_aug,
    )
    test_loader = DataLoader(
        test_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True
    )

    for fold in range(5):
        LOGGER.info(f"========== model: {CONFIG.MODEL_NAME} fold: {fold} inference ==========")
        model = PawpularModel(model_name=CONFIG.MODEL_NAME)
        model.to(CONFIG.device)
        if torch.cuda.device_count()>1:
            model=nn.DataParallel(model)
            model.load_state_dict(torch.load(CONFIG.OUTPUT_DIR / f"{CONFIG.MODEL_NAME}_{fold}_best.pth")["model"])
        else:
            model.load_state_dict(fix_model_state_dict(torch.load(CONFIG.OUTPUT_DIR / f"{CONFIG.MODEL_NAME}_{fold}_best.pth")["model"]))
        model.eval()
        preds = []
        for i, data in tqdm(enumerate(test_loader), total=len(test_loader)):
            img, feature ,target = data['image'], data['features'], data['targets']
            img = img.to(CONFIG.device)
            feature = feature.to(CONFIG.device)
            target = target.to(CONFIG.device)
            with torch.no_grad():
                y_preds = model(img, feature)
            preds.append(y_preds.to("cpu").numpy())
        preds = np.concatenate(preds)
        predictions.append(preds)
    predictions = np.mean(predictions, axis=0)
    return predictions * 100

In [13]:
def main():
    # Training
    oof_df = pd.DataFrame()
    if CONFIG.training_step:
        for fold in range(CONFIG.fold):
            _oof_df = train_loop(train_df, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)
        # Save OOF result
        oof_df.to_csv(CONFIG.OUTPUT_DIR / "oof_df.csv", index=False)
    else:
        oof_df = pd.read_csv(CONFIG.OUTPUT_DIR / "oof_df.csv")
    # CV result
    LOGGER.info(f"========== CV ==========")
    get_result(oof_df)
    # Inference
    predictions = inference()
    # submission
    submission = test_df.copy()
    submission["Pawpularity"] = predictions
    submission = submission[["Id", "Pawpularity"]]
    submission.to_csv(CONFIG.OUTPUT_DIR / "submission.csv", index=False)

In [14]:
if __name__ == "__main__":
    main()

========== fold: 0 training ==========
  0%|                                                                                                                                                                              | 0/991 [00:01<?, ?it/s]


RuntimeError: Caught RuntimeError in replica 0 on device 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/parallel_apply.py", line 61, in _worker
    output = module(*input, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/tmp/ipykernel_647259/1031293254.py", line 11, in forward
    x = self.model(image)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result = self.forward(*input, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/timm/models/efficientnet.py", line 477, in forward
    x = self.forward_features(x)
  File "/opt/conda/lib/python3.7/site-packages/timm/models/efficientnet.py", line 470, in forward_features
    x = self.blocks(x)
  File "/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py", line 727, in _call_impl
    result =